<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/2_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Flask 處理 /download/{file_path}

#### 1-1. Flask 如何處理

- 當用戶點擊 下載連結 時，瀏覽器會發送請求，例如：GET /download/202502/202502_A.txt
- 這時候：
  1. file_path = "202502/202502_A.txt"
  2. full_path = os.path.join(BASE_PATH, file_path)
  🔹 結果 → ../../data_source/update/202502/202502_A.txt
  3. send_file(full_path, as_attachment=True) 讓檔案下載，而不是在瀏覽器打開。

In [ ]:
from flask import Flask, send_file, abort
import os

app = Flask(__name__)

# 檔案根目錄


@app.route('/download/<path:file_path>')
def download_file(file_path):
    # 檔案根目錄
    BASE_PATH = '../../data_source/update'
    # 拼接完整檔案路徑
    full_path = os.path.join(BASE_PATH, file_path)

    # 確保檔案存在
    if os.path.exists(full_path):
        return send_file(full_path, as_attachment=True)  # 強制下載
    else:
        return abort(404)  # 若檔案不存在則回傳 404

if __name__ == '__main__':
    app.run(debug=True)


#### 1-2.  Dash 產生正確下載連結

確保 href 指向 /download/{file_path}，而不是 ../../data_source/update/{file_path}，因為 /download/ 會自動加上 BASE_PATH。

In [ ]:
def generate_file_links(file_string):
    if not file_string:
        return ""

    files = file_string.split(', ')
    links = []

    for file in files:
        parts = file.split('_', 1)
        if len(parts) > 1:
            display_name = parts[1]  # 取 _ 之後的字眼
            month = parts[0]  # 取 _ 之前的月份字串
            file_id = f"{month}/{file}"  # 例如 "202502/202502_A.txt"

            # 正確的 Flask 下載 URL
            link = f'<a href="/download/{file_id}" download>{display_name}</a>'
            links.append(link)

    return "  \n".join(links)


 **總結**

- /download/{file_path} 只是 URL，實際檔案路徑是 BASE_PATH + file_path
- Flask send_file 負責找到檔案並強制下載
- Dash 內部 href 連結應該指向 /download/{file_path}，而不是直接寫完整路徑

這樣 .txt、.csv、.png 這些檔案就能正常下載了

## 2. 有下載該檔案，但格式都會變成html

Reason : 瀏覽器錯誤地將下載的檔案當作 HTML 處理，通常是因為 Flask 在回應時，*Content-Type 沒有正確設定，或者檔案的 MIME 類型未正確識別*。

ans : 明確指定 mimetype

In [ ]:
from flask import Flask, send_file, abort, mimetypes
import os

app = Flask(__name__)

# 檔案根目錄
BASE_PATH = '../../data_source/update'

@app.route('/download/<path:file_path>')
def download_file(file_path):
    # 拼接完整檔案路徑
    full_path = os.path.join(BASE_PATH, file_path)

    # 確保檔案存在
    if not os.path.exists(full_path):
        return abort(404)  # 若檔案不存在則回傳 404

    # 嘗試根據檔案副檔名判斷 MIME 類型
    mime_type, _ = mimetypes.guess_type(full_path)
    if mime_type is None:
        mime_type = "application/octet-stream"  # 預設為二進位流，防止被當成 HTML

    print(f"✅ 下載檔案: {full_path} (MIME: {mime_type})")

    return send_file(full_path, as_attachment=True, mimetype=mime_type)  # 強制下載並確保格式正確

if __name__ == '__main__':
    app.run(debug=True)


1. mimetypes.guess_type(full_path) 作用 (自動偵測檔案的 MIME 類型)，確保下載時，瀏覽器不會錯誤解析成 HTML :

*   txt → text/plain
*   csv → text/csv
*   png → image/png
*   xlsx → application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
*   docx → application/vnd.openxmlformats-officedocument.wordprocessingml.document
*   doc → application/msword

2. application/octet-stream（預設值）

  如果 MIME 類型無法被偵測，就使用 application/octet-stream，這樣瀏覽器會強制下載檔案，而不會嘗試開啟它

3. as_attachment=True

  讓瀏覽器正確地下載檔案，而不是直接打開。

## 3. 用 callback來下載

In [ ]:
import dash
from dash import dcc, html, dash_table, Input, Output, State
import pandas as pd
import os

app = dash.Dash(__name__)

# 原始數據（每個 Cell 只包含 1 個檔案）
data = [
    [1, "H02U", 'DH240P23.00', 23, '202502_A.txt'],
    [2, "H02D", 'DH240P11.00', 23, '202502_B.xlsx'],
    [3, "H03F", 'DH240P10.00', 21, '202502_C.png'],
    [4, "D03F", 'CH240P10.00', 11, '202502_D.csv']
]

# 轉換為 DataFrame
df = pd.DataFrame(data, columns=["Issue Number", "Chart Name", "Lot", "Count", "File"])

# 轉換 File 欄位為 Markdown 超連結格式
def generate_file_link(file_string):
    if not file_string:
        return ""

    parts = file_string.split('_', 1)
    if len(parts) > 1:
        display_name = parts[1]  # 取 _ 之後的字眼
        month = parts[0]  # 取 _ 之前的月份字串
        file_id = f"{month}/{file_string}"  # 構造唯一識別符

        # Markdown 格式的超連結（會變成藍色字體，帶底線）
        return f"[{display_name}](download://{file_id})"

    return file_string

df["File"] = df["File"].apply(generate_file_link)

app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[
            {"name": col, "id": col, "presentation": "markdown"} if col == "File" else {"name": col, "id": col}
            for col in df.columns
        ],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
    ),
    dcc.Download(id="download-component")
])

# 監聽 active_cell，點擊時下載對應檔案
@app.callback(
    Output("download-component", "data"),
    Input("table", "active_cell"),
    State("table", "data"),
    prevent_initial_call=True
)
def trigger_download(active_cell, table_data):
    if not active_cell:
        return dash.no_update

    row = active_cell["row"]
    col = active_cell["column_id"]

    # 確保點擊的是 File 欄位
    if col != "File":
        return dash.no_update

    file_link = table_data[row]["File"]
    if not file_link:
        return dash.no_update

    # 取得檔案路徑
    file_id = file_link.split('](')[1].rstrip(')')  # 從 Markdown 超連結格式提取檔案 ID
    file_path = f"../../data_source/{file_id}"

    if os.path.exists(file_path):
        return dcc.send_file(file_path)

    return dash.no_update

if __name__ == '__main__':
    app.run_server(debug=True)
